In [107]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
import re

In [113]:
#create full dataframe with all stats
def createWRDF(dfwrA, dfwrB, dfwrnext):
    dfwrA = cleanFP(dfwrA)
    dfwrB = cleanPFR(dfwrB)
    dfwr = combine(dfwrA, dfwrB)
    dfwr = addNextYrFpts(dfwr, dfwrnext)

#add fpts/g for next season
def addNextYrFpts(dfwrcurr, dfwrnext):
    dfwrcurr = pd.merge(dfwrcurr, dfwrnext[['Player', 'Fpts/g']], left_on='Player', right_on='Player', how='inner', suffixes=('', '_nextYr'))
    return dfwrcurr

#add advanced stats from fantasy pros
def cleanFP(dfwr):
    dfwr['Player'] = dfwr['Player'].str.split("(").str[0].str.strip()
    dfwr = dfwr.drop(columns=['AIR', 'AIR/R', 'G', 'REC', 'YDS'])
    return dfwr
#add basic stats from pro football reference
def cleanPFR(dfwr):
    dfwr['Player'] = dfwr['Player'].str.replace(re.escape('*'), '', regex=True)
    dfwr['Player'] = dfwr['Player'].str.replace(re.escape('+'), '', regex=True).str.strip()
    dfwr['Fpts/G'] = dfwr.apply(lambda row: (row.Rec + row.Yds / 10 + row.TD * 6 + row.Fmb * - 2) / row.G, axis=1)
    dfwr = dfwr.drop(columns=['Tm', 'Pos', 'Y/R', 'Tgt', 'Lng', 'Rk'])
    return dfwr
#combine advanced and basic stats into one dataframe
def combine(dfwrA, dfwrB):
    dfwr = pd.merge(dfwrA, dfwrB, left_on='Player', right_on='Player', how='inner')
    return dfwr

dfwr22A = pd.read_excel(r"C:\Users\luker\Downloads\Python\WR22.xlsx", 'WR22A')
dfwr22A = cleanFP(dfwr22A)
dfwr22B = pd.read_excel(r"C:\Users\luker\Downloads\Python\WR22.xlsx", 'WR22B')
dfwr22B = cleanPFR(dfwr22B)
dfwr22 = combine(dfwr22A, dfwr22B)
dfwr22.columns

Index(['Rank', 'Player', 'Y/R', 'YBC', 'YBC/R', 'YAC', 'YAC/R', 'YACON',
       'YACON/R', 'BRKTKL', 'TGT', '% TM', 'CATCHABLE', 'DROP', 'RZ TGT',
       '10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'LNG', 'Age',
       'G', 'GS', 'Rec', 'Ctch%', 'Yds', 'TD', '1D', 'Succ%', 'Y/Tgt', 'R/G',
       'Y/G', 'Fmb', 'Fpts/G'],
      dtype='object')